In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Zelle 1: Install

In [2]:
!pip -q install -U transformers datasets accelerate scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 149.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 52.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.0 which is incompatible.
cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, but you have pandas 3.0.0 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.0 whi

In [3]:
!pip -q uninstall -y torch torchvision torchaudio
!pip -q install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 202.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 341.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 380.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 300.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 406.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 220.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 160.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 292.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 213.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 291.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 284.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 258.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

Zelle 2: Imports

In [4]:
import json
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

Zelle 3: Pfade + Modell

In [5]:
DEPT_CSV = "/content/drive/MyDrive/BuzzwordLearner/data/department-v2.csv"
CV_ANN   = "/content/drive/MyDrive/BuzzwordLearner/data/linkedin-cvs-annotated.json"
SEN_CSV = "/content/drive/MyDrive/BuzzwordLearner/data/seniority-v2.csv"

MODEL_NAME = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 32
SEED = 42

print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

CUDA: True
GPU: NVIDIA L4


Zelle 4: Train Lookup laden

In [6]:
dept_df = pd.read_csv(DEPT_CSV).dropna(subset=["text", "label"]).copy()
dept_df["text"]  = dept_df["text"].astype(str).str.strip()
dept_df["label"] = dept_df["label"].astype(str).str.strip()

print("Train rows:", len(dept_df))
print(dept_df["label"].value_counts())

Train rows: 10145
label
Marketing                 4295
Sales                     3328
Information Technology    1305
Business Development       620
Project Management         201
Consulting                 167
Administrative              83
Other                       42
Purchasing                  40
Customer Support            33
Human Resources             31
Name: count, dtype: int64


In [7]:
sen_df = pd.read_csv(SEN_CSV).dropna(subset=["text", "label"]).copy()
sen_df["text"]  = sen_df["text"].astype(str).str.strip()
sen_df["label"] = sen_df["label"].astype(str).str.strip()

print("Seniority TRAIN rows:", len(sen_df))
print(sen_df["label"].value_counts())

Seniority TRAIN rows: 9428
label
Senior        3733
Lead          3546
Director       984
Management     756
Junior         409
Name: count, dtype: int64


Zelle 5: Eval laden (nur messen)

In [8]:
with open(CV_ANN, "r", encoding="utf-8") as f:
    ann = json.load(f)

positions = [p for cv in ann for p in cv]
eval_df = pd.DataFrame(positions)

eval_df["status"] = eval_df["status"].astype(str).str.upper()
eval_df = eval_df[eval_df["status"] == "ACTIVE"].copy()

eval_df["title"] = eval_df["position"].astype(str).str.strip()
eval_df["department"] = eval_df["department"].astype(str).str.strip()
eval_df["seniority"]  = eval_df["seniority"].astype(str).str.strip()

# wichtig: NICHT mehr nur ["title","department"] auswählen
eval_df = eval_df[["title", "department", "seniority"]].dropna(subset=["title"]).copy()

print("Eval ACTIVE rows:", len(eval_df))
print("\nDepartment eval dist:\n", eval_df["department"].value_counts())
print("\nSeniority eval dist:\n", eval_df["seniority"].value_counts())

Eval ACTIVE rows: 623

Department eval dist:
 department
Other                     344
Information Technology     62
Sales                      46
Consulting                 39
Project Management         39
Marketing                  22
Business Development       20
Human Resources            16
Purchasing                 15
Administrative             14
Customer Support            6
Name: count, dtype: int64

Seniority eval dist:
 seniority
Professional    216
Management      192
Lead            125
Senior           44
Director         34
Junior           12
Name: count, dtype: int64


Zelle 6: Tokenizer + Helper

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)

collator = DataCollatorWithPadding(tokenizer=tokenizer)

def make_ds(df, text_col, label_col=None):
    data = {"text": df[text_col].astype(str).tolist()}
    if label_col is not None:
        data["labels"] = df[label_col].astype(int).tolist()
    ds = Dataset.from_dict(data)
    return ds.map(tokenize, batched=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
    }

def make_trainer(task_name, num_labels, train_ds, val_ds, lr=2e-5, batch_size=512, epochs=20, patience=3):
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels
    )

    args = TrainingArguments(
        output_dir=f"./out_{task_name}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        save_safetensors=True,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        learning_rate=lr,
        lr_scheduler_type="linear",
        warmup_ratio=0.06,
        weight_decay=0.01,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        logging_strategy="epoch",
        logging_steps=0,
        seed=SEED,
        report_to="none",
        fp16=False,
        bf16=torch.cuda.is_available(),

        dataloader_num_workers=8,
        dataloader_pin_memory=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)]
    )
    return trainer

In [10]:
def compute_class_weights(y_int, num_classes):
    counts = np.bincount(y_int, minlength=num_classes)
    total = counts.sum()
    weights = total / (num_classes * counts)
    return weights

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [11]:
import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    """
    Multi-class Focal Loss.
    alpha: Tensor shape [num_classes] oder None (für class weights)
    gamma: Fokus-Parameter (typisch 1.5 - 2.0)
    """
    def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        # logits: [B, C], targets: [B]
        log_probs = F.log_softmax(logits, dim=-1)              # [B, C]
        probs = torch.exp(log_probs)                           # [B, C]

        # log_prob für die wahre Klasse je Sample
        targets = targets.long()
        log_pt = log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)  # [B]
        pt = probs.gather(1, targets.unsqueeze(1)).squeeze(1)          # [B]

        # Alpha weighting (class weights)
        if self.alpha is not None:
            at = self.alpha.to(logits.device).gather(0, targets)       # [B]
        else:
            at = 1.0

        loss = -at * ((1 - pt) ** self.gamma) * log_pt                 # [B]

        if self.reduction == "mean":
            return loss.mean()
        if self.reduction == "sum":
            return loss.sum()
        return loss


class FocalTrainer(Trainer):
    def __init__(self, alpha=None, gamma=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal = FocalLoss(alpha=alpha, gamma=gamma)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.focal(logits, labels)
        return (loss, outputs) if return_outputs else loss

STAGE 1: Other vs NotOther

Zelle 7: Stage1 Dataset bauen (Binary)

In [12]:
stage1_df = dept_df.copy()
stage1_df["bin_label"] = (stage1_df["label"] == "Other").astype(int)  # 1=Other, 0=NotOther

train1, val1 = train_test_split(
    stage1_df,
    test_size=0.2,
    random_state=SEED,
    stratify=stage1_df["bin_label"]
)

train1_ds = make_ds(train1, "text", "bin_label")
val1_ds   = make_ds(val1, "text", "bin_label")

print("Stage1 train:", len(train1), "val:", len(val1))
print(train1["bin_label"].value_counts())

Map:   0%|          | 0/8116 [00:00<?, ? examples/s]

Map:   0%|          | 0/2029 [00:00<?, ? examples/s]

Stage1 train: 8116 val: 2029
bin_label
0    8082
1      34
Name: count, dtype: int64


Zelle 8: Stage1 train (optional LR Sweep)

In [13]:
# ===== ZELLE 8: STAGE 1 TRAIN (Other vs NotOther) - FIXED + STABLE SETTINGS =====

w1 = torch.tensor(compute_class_weights(train1["bin_label"].values, 2), dtype=torch.float)
print("Stage1 class weights:", w1.tolist())
print("Stage1 train label counts:\n", train1["bin_label"].value_counts())

model1 = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

args1 = TrainingArguments(
    output_dir="./out_dept_stage1_other_vs_not",
    eval_strategy="epoch",   # FIX (war eval_strategy)
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_ratio=0.06,
    weight_decay=0.01,

    per_device_train_batch_size=256,
    per_device_eval_batch_size=516,
    gradient_accumulation_steps=4,  # effektiv ~256
    num_train_epochs=50,
    logging_strategy="epoch",
    logging_steps=0,
    seed=SEED,
    report_to="none",
    fp16=False,
    bf16=torch.cuda.is_available(),

    dataloader_num_workers=8,
    dataloader_pin_memory=True,
)

stage1_trainer = WeightedTrainer(
    class_weights=w1,
    model=model1,
    args=args1,
    train_dataset=train1_ds,
    eval_dataset=val1_ds,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
)

stage1_trainer.train()
print("Stage1 val:", stage1_trainer.evaluate())

Stage1 class weights: [0.5021034479141235, 119.35294342041016]
Stage1 train label counts:
 bin_label
0    8082
1      34
Name: count, dtype: int64


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-multilingual-cased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,2.873875,0.697726,0.004929,0.004920,0.002000
2,2.767169,0.669636,0.538196,0.354685,0.696097
3,2.526791,0.606100,0.960079,0.534740,0.976084
4,2.073024,0.476433,0.986693,0.650487,0.990579
5,1.459597,0.312155,0.991129,0.733058,0.993457
6,0.818267,0.166192,0.991129,0.733058,0.993457
7,0.421110,0.095253,0.993593,0.774249,0.995019
8,0.203355,0.169839,0.997043,0.849257,0.997337
9,0.103880,0.263533,0.998521,0.911394,0.998565
10,0.059315,0.261142,0.998521,0.911394,0.998565


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


Stage1 val: {'eval_loss': 0.27297213673591614, 'eval_accuracy': 0.9995071463775259, 'eval_f1_macro': 0.9665429961249897, 'eval_f1_weighted': 0.999490778874918, 'eval_runtime': 0.4599, 'eval_samples_per_second': 4411.958, 'eval_steps_per_second': 8.698, 'epoch': 18.0}


STAGE 2: Multi-class für NotOther

Zelle 9: Stage2 Dataset bauen (nur NotOther)

In [14]:
stage2_df = dept_df[dept_df["label"] != "Other"].copy()

le2 = LabelEncoder()
stage2_df["y"] = le2.fit_transform(stage2_df["label"])

train2, val2 = train_test_split(
    stage2_df,
    test_size=0.2,
    random_state=SEED,
    stratify=stage2_df["y"]
)

train2_ds = make_ds(train2, "text", "y")
val2_ds   = make_ds(val2, "text", "y")

print("Stage2 train:", len(train2), "val:", len(val2))
print("Stage2 classes:", list(le2.classes_))
print(stage2_df["label"].value_counts())

Map:   0%|          | 0/8082 [00:00<?, ? examples/s]

Map:   0%|          | 0/2021 [00:00<?, ? examples/s]

Stage2 train: 8082 val: 2021
Stage2 classes: ['Administrative', 'Business Development', 'Consulting', 'Customer Support', 'Human Resources', 'Information Technology', 'Marketing', 'Project Management', 'Purchasing', 'Sales']
label
Marketing                 4295
Sales                     3328
Information Technology    1305
Business Development       620
Project Management         201
Consulting                 167
Administrative              83
Purchasing                  40
Customer Support            33
Human Resources             31
Name: count, dtype: int64


Zelle 10: Stage2 train

In [16]:
# ===== STAGE 2 TRAIN (NotOther Multi-class) mit FOCAL LOSS + class weights =====

# class weights (alpha) aus Train2
w2 = torch.tensor(compute_class_weights(train2["y"].values, len(le2.classes_)), dtype=torch.float)
print("Stage2 class weights (first 10):", w2[:10].tolist())
print("Stage2 train label counts:\n", train2["label"].value_counts().head(20))

model2 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(le2.classes_)
)

# focal gamma: typischer Start
FOCAL_GAMMA = 2.0

args2 = TrainingArguments(
    output_dir="./out_dept_stage2_notother_focal",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    learning_rate=1e-5,                  # du meintest 1e-5 war besser
    lr_scheduler_type="linear",
    warmup_ratio=0.06,
    weight_decay=0.01,

    per_device_train_batch_size=256,
    per_device_eval_batch_size=516,
    gradient_accumulation_steps=4,       # effektiv 256
    num_train_epochs=50,

    logging_strategy="epoch",
    logging_steps=0,
    seed=SEED,
    report_to="none",

    bf16=torch.cuda.is_available(),      # auf L4 ok
    fp16=False,

    dataloader_num_workers=8,
    dataloader_pin_memory=True,
)

stage2_trainer = FocalTrainer(
    alpha=w2, gamma=FOCAL_GAMMA,
    model=model2,
    args=args2,
    train_dataset=train2_ds,
    eval_dataset=val2_ds,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

stage2_trainer.train()
print("Stage2 val:", stage2_trainer.evaluate())

Stage2 class weights (first 10): [12.245454788208008, 1.6294355392456055, 6.031343460083008, 31.08461570739746, 32.327999114990234, 0.7741379141807556, 0.23521536588668823, 5.019876003265381, 25.256250381469727, 0.3036063015460968]
Stage2 train label counts:
 label
Marketing                 3436
Sales                     2662
Information Technology    1044
Business Development       496
Project Management         161
Consulting                 134
Administrative              66
Purchasing                  32
Customer Support            26
Human Resources             25
Name: count, dtype: int64


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-multilingual-cased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,7.443303,1.870157,0.007917,0.002629,0.007830
2,7.407472,1.848400,0.165265,0.072468,0.206146
3,7.260073,1.803671,0.350816,0.136100,0.308944
4,7.034908,1.723316,0.347848,0.231372,0.326990
5,6.642938,1.597282,0.442355,0.293354,0.472920
6,6.103435,1.422262,0.552697,0.351973,0.600684
7,5.364611,1.210633,0.673924,0.425505,0.720746
8,4.617039,1.000839,0.805542,0.573330,0.835312
9,3.809736,0.800246,0.853538,0.646463,0.872855
10,3.057404,0.620972,0.876794,0.695060,0.890846


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


Stage2 val: {'eval_loss': 0.07513265311717987, 'eval_accuracy': 0.9960415635823849, 'eval_f1_macro': 0.9838988192386271, 'eval_f1_weighted': 0.9960232161117657, 'eval_runtime': 0.4922, 'eval_samples_per_second': 4106.197, 'eval_steps_per_second': 8.127, 'epoch': 40.0}


In [17]:
# LabelEncoder nur für Seniority
le_s = LabelEncoder()
sen_df["y"] = le_s.fit_transform(sen_df["label"])

train_s, val_s = train_test_split(
    sen_df,
    test_size=0.2,
    random_state=SEED,
    stratify=sen_df["y"]
)

train_s_ds = make_ds(train_s, "text", "y")
val_s_ds   = make_ds(val_s, "text", "y")

# Class weights
w_s = torch.tensor(
    compute_class_weights(train_s["y"].values, len(le_s.classes_)),
    dtype=torch.float
)

print("Seniority classes:", list(le_s.classes_))
print("Train dist:\n", train_s["label"].value_counts())
print("Class weights:", w_s.tolist())

# Trainer
model_s = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(le_s.classes_)
)

args_s = TrainingArguments(
    output_dir="./out_seniority",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,

    learning_rate=1e-5,              # bei dir oft besser
    lr_scheduler_type="linear",
    warmup_ratio=0.06,
    weight_decay=0.01,

    per_device_train_batch_size=256,
    per_device_eval_batch_size=516,
    gradient_accumulation_steps=8,   # effektiv 256
    num_train_epochs=50,

    logging_strategy="epoch",
    seed=SEED,
    report_to="none",
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=8,
    dataloader_pin_memory=True,
)

sen_trainer = WeightedTrainer(
    class_weights=w_s,
    model=model_s,
    args=args_s,
    train_dataset=train_s_ds,
    eval_dataset=val_s_ds,
    processing_class=tokenizer,   # WICHTIG: neuer Transformers-Standard
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

sen_trainer.train()
print("Seniority VAL:", sen_trainer.evaluate())

Map:   0%|          | 0/7542 [00:00<?, ? examples/s]

Map:   0%|          | 0/1886 [00:00<?, ? examples/s]

Seniority classes: ['Director', 'Junior', 'Lead', 'Management', 'Senior']
Train dist:
 label
Senior        2986
Lead          2837
Director       787
Management     605
Junior         327
Name: count, dtype: int64
Class weights: [1.9166455268859863, 4.612843990325928, 0.531688392162323, 2.493223190307617, 0.5051574110984802]


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-multilingual-cased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


Seniority VAL: {'eval_loss': 0.08555778115987778, 'eval_accuracy': 0.9925768822905621, 'eval_f1_macro': 0.989877182822925, 'eval_f1_weighted': 0.9925528428200731, 'eval_runtime': 0.5096, 'eval_samples_per_second': 3701.05, 'eval_steps_per_second': 7.85, 'epoch': 42.0}


In [18]:
# ===== ZELLE 11: STAGE 1 THRESHOLD SWEEP (Other vs NotOther) =====
# Ziel: beste Confidence (TH) anhand Macro-F1 finden und für Inference speichern

valid_labels = set(dept_df["label"].unique())
eval_s1 = eval_df[eval_df["department"].isin(valid_labels)].copy()

# y_true: 1 = Other, 0 = NotOther
y_true = (eval_s1["department"].astype(str) == "Other").astype(int).values

# Stage 1 probabilities
ds_s1 = Dataset.from_dict({
    "text": eval_s1["title"].astype(str).tolist()
}).map(tokenize, batched=True)

logits = stage1_trainer.predict(ds_s1).predictions
prob_other = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()

# Thresholds to test
ths = [0.20, 0.25, 0.30, 0.40, 0.50, 0.60, 0.70, 0.75, 0.80]
rows = []

for TH in ths:
    y_pred = (prob_other >= TH).astype(int)

    rows.append({
        "TH": TH,
        "pred_other_rate": float(np.mean(y_pred == 1)),
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_f1": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_other": f1_score(y_true, y_pred, pos_label=1, zero_division=0),
        "f1_notother": f1_score(y_true, y_pred, pos_label=0, zero_division=0),
        "precision_other": precision_score(y_true, y_pred, pos_label=1, zero_division=0),
        "recall_other": recall_score(y_true, y_pred, pos_label=1, zero_division=0),
    })

# Results table
res = pd.DataFrame(rows).sort_values("macro_f1", ascending=False)

print("\n=== STAGE 1 THRESHOLD RESULTS (sorted by macro_f1) ===")
display(res)

# Best threshold
best_TH = float(res.iloc[0]["TH"])
BEST_TH_STAGE1 = best_TH  # <-- WICHTIG: für Zelle 11 (Two-Stage Inference)

print("Best TH by macro_f1:", BEST_TH_STAGE1)

# Detailed report for best TH
y_pred_best = (prob_other >= BEST_TH_STAGE1).astype(int)

print(f"\n=== STAGE 1 DETAILED REPORT @ TH={BEST_TH_STAGE1:.2f} ===")
print("Accuracy :", accuracy_score(y_true, y_pred_best))
print("Macro F1 :", f1_score(y_true, y_pred_best, average="macro", zero_division=0))
print("\nClassification report:\n")
print(classification_report(y_true, y_pred_best, digits=4, zero_division=0))

Map:   0%|          | 0/623 [00:00<?, ? examples/s]


=== STAGE 1 THRESHOLD RESULTS (sorted by macro_f1) ===


,TH,pred_other_rate,accuracy,macro_f1,f1_other,f1_notother,precision_other,recall_other
0,0.20,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
1,0.25,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
2,0.30,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
3,0.40,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
4,0.50,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
5,0.60,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
6,0.70,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
7,0.75,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256
8,0.80,0.016051,0.457464,0.333137,0.045198,0.621076,0.8,0.023256


Best TH by macro_f1: 0.2

=== STAGE 1 DETAILED REPORT @ TH=0.20 ===
Accuracy : 0.45746388443017655
Macro F1 : 0.33313698664842545

Classification report:

              precision    recall  f1-score   support

           0     0.4519    0.9928    0.6211       279
           1     0.8000    0.0233    0.0452       344

    accuracy                         0.4575       623
   macro avg     0.6259    0.5080    0.3331       623
weighted avg     0.6441    0.4575    0.3031       623



In [19]:
# ===== Stage 2 EVAL on eval_df (only NotOther classes) =====
valid_stage2 = set(le2.classes_)
eval_s2 = eval_df[eval_df["department"].isin(valid_stage2)].copy()

y_true_s2 = eval_s2["department"].astype(str).values
y_true_s2_ids = le2.transform(y_true_s2)

ds_s2 = Dataset.from_dict({"text": eval_s2["title"].astype(str).tolist()}).map(tokenize, batched=True)
logits2 = stage2_trainer.predict(ds_s2).predictions
y_pred_s2_ids = np.argmax(logits2, axis=-1)
y_pred_s2 = le2.inverse_transform(y_pred_s2_ids)

print("\n=== STAGE 2 EVAL (NotOther Multi-class) on eval_df ===")
print("Accuracy       :", accuracy_score(y_true_s2, y_pred_s2))
print("Macro Precision:", precision_score(y_true_s2, y_pred_s2, average="macro", zero_division=0))
print("Macro Recall   :", recall_score(y_true_s2, y_pred_s2, average="macro", zero_division=0))
print("Macro F1       :", f1_score(y_true_s2, y_pred_s2, average="macro", zero_division=0))
print("Weighted F1    :", f1_score(y_true_s2, y_pred_s2, average="weighted", zero_division=0))
print("\nReport:\n", classification_report(y_true_s2, y_pred_s2, digits=4, zero_division=0))

Map:   0%|          | 0/279 [00:00<?, ? examples/s]


=== STAGE 2 EVAL (NotOther Multi-class) on eval_df ===
Accuracy       : 0.6236559139784946
Macro Precision: 0.667903689877374
Macro Recall   : 0.5203958920929468
Macro F1       : 0.5510529771454784
Weighted F1    : 0.6155559753210169

Report:
                         precision    recall  f1-score   support

        Administrative     0.5000    0.4286    0.4615        14
  Business Development     0.5455    0.3000    0.3871        20
            Consulting     0.5750    0.5897    0.5823        39
      Customer Support     1.0000    0.1667    0.2857         6
       Human Resources     0.8889    0.5000    0.6400        16
Information Technology     0.5474    0.8387    0.6624        62
             Marketing     0.5238    0.5000    0.5116        22
    Project Management     0.7179    0.7179    0.7179        39
            Purchasing     0.5385    0.4667    0.5000        15
                 Sales     0.8421    0.6957    0.7619        46

              accuracy                         0.

INFERENCE PIPELINE (Stage1 -> Stage2)

Zelle 11: Zwei-Stufen Vorhersage auf eval_df + Metriken

In [20]:
# ===== ZELLE 11: INFERENCE + TH2 SWEEP (komplett, robust) =====

# 1) Eval-Subset: nur Labels, die Stage2 kennt + "Other"
valid_labels = set(le2.classes_) | {"Other"}
eval_use = eval_df[eval_df["department"].isin(valid_labels)].copy()
print("Eval used:", len(eval_use))

# 2) Stage1 probs (Other vs NotOther)
ds_s1 = Dataset.from_dict({"text": eval_use["title"].astype(str).tolist()}).map(tokenize, batched=True)
logits1 = stage1_trainer.predict(ds_s1).predictions
p1_prob_other = torch.softmax(torch.tensor(logits1), dim=-1)[:, 1].numpy()

# 3) TH für Stage1 (nimmt BEST_TH_STAGE1 falls gesetzt, sonst 0.5)
TH1 = float(globals().get("BEST_TH_STAGE1", 0.5))
print("Using TH1 (Stage1):", TH1)

pred_is_other = p1_prob_other >= TH1

# 4) Stage2 prediction nur für NotOther (die durch Stage1 nicht geroutet wurden)
eval_notother = eval_use.loc[~pred_is_other].copy()

# Default: alles Other
base_pred = np.array(["Other"] * len(eval_use), dtype=object)

if len(eval_notother) > 0:
    eval_stage2_ds = Dataset.from_dict({"text": eval_notother["title"].astype(str).tolist()}).map(tokenize, batched=True)
    p2_logits = stage2_trainer.predict(eval_stage2_ds).predictions
    p2_probs = torch.softmax(torch.tensor(p2_logits), dim=-1).numpy()
    max_conf = p2_probs.max(axis=-1)

    p2_ids = np.argmax(p2_probs, axis=-1)
    p2_labels_base = le2.inverse_transform(p2_ids)
else:
    p2_probs = None
    max_conf = np.array([], dtype=float)
    p2_labels_base = np.array([], dtype=object)

# 5) TH2 Sweep: confidence gate für Stage2 (unter TH2 => "Other")
th2s = [0.20, 0.25, 0.30, 0.40, 0.50, 0.60, 0.70, 0.75, 0.80]
rows = []

y_true = eval_use["department"].astype(str).values

for TH2 in th2s:
    y_pred = base_pred.copy()

    if len(eval_notother) > 0:
        p2_labels = p2_labels_base.copy()
        p2_labels = np.where(max_conf < TH2, "Other", p2_labels)
        y_pred[~pred_is_other] = p2_labels

    rows.append({
        "TH1": TH1,
        "TH2": TH2,
        "macro_f1": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted", zero_division=0),
        "accuracy": accuracy_score(y_true, y_pred),
        "pred_other_rate": float(np.mean(y_pred == "Other")),
        "n_routed_stage1_other": int(np.sum(pred_is_other)),
        "n_stage2_used": int(np.sum(~pred_is_other)),
    })

res_th2 = pd.DataFrame(rows).sort_values("macro_f1", ascending=False)
print("\n=== TH2 SWEEP (Stage2 confidence gate) sorted by macro_f1 ===")
display(res_th2)

BEST_TH2 = float(res_th2.iloc[0]["TH2"])
print("Best TH2:", BEST_TH2)

# 6) Final predictions using BEST_TH2
y_pred = base_pred.copy()
if len(eval_notother) > 0:
    p2_labels = np.where(max_conf < BEST_TH2, "Other", p2_labels_base.copy())
    y_pred[~pred_is_other] = p2_labels

print("\n=== Two-stage FINAL EVAL (with TH1 + BEST_TH2) ===")
print("Accuracy       :", accuracy_score(y_true, y_pred))
print("Macro Precision:", precision_score(y_true, y_pred, average="macro", zero_division=0))
print("Macro Recall   :", recall_score(y_true, y_pred, average="macro", zero_division=0))
print("Macro F1       :", f1_score(y_true, y_pred, average="macro", zero_division=0))
print("Weighted F1    :", f1_score(y_true, y_pred, average="weighted", zero_division=0))

print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4, zero_division=0))

preview = eval_use[["title", "department"]].copy()
preview["pred"] = y_pred
preview["p_other"] = p1_prob_other
preview["routed_other_stage1"] = pred_is_other

Eval used: 623


Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Using TH1 (Stage1): 0.2


Map:   0%|          | 0/613 [00:00<?, ? examples/s]


=== TH2 SWEEP (Stage2 confidence gate) sorted by macro_f1 ===


,TH1,TH2,macro_f1,weighted_f1,accuracy,pred_other_rate,n_routed_stage1_other,n_stage2_used
5,0.2,0.60,0.535894,0.685936,0.677368,0.605136,10,613
6,0.2,0.70,0.532561,0.702333,0.717496,0.693419,10,613
7,0.2,0.75,0.528880,0.700724,0.723917,0.731942,10,613
8,0.2,0.80,0.510533,0.684561,0.714286,0.765650,10,613
4,0.2,0.50,0.503253,0.636260,0.617978,0.510433,10,613
3,0.2,0.40,0.469594,0.580484,0.552167,0.388443,10,613
2,0.2,0.30,0.418268,0.453661,0.434992,0.208668,10,613
1,0.2,0.25,0.395793,0.394388,0.391653,0.152488,10,613
0,0.2,0.20,0.371203,0.302698,0.335474,0.067416,10,613


Best TH2: 0.6

=== Two-stage FINAL EVAL (with TH1 + BEST_TH2) ===
Accuracy       : 0.6773675762439807
Macro Precision: 0.7208496295661657
Macro Recall   : 0.46608594215874005
Macro F1       : 0.5358938030969786
Weighted F1    : 0.6859363567141729

Classification report:
                        precision    recall  f1-score   support

        Administrative     0.0233    0.0714    0.0351        14
  Business Development     0.3750    0.3000    0.3333        20
            Consulting     0.9048    0.4872    0.6333        39
      Customer Support     1.0000    0.1667    0.2857         6
       Human Resources     0.8889    0.5000    0.6400        16
Information Technology     0.6027    0.7097    0.6519        62
             Marketing     0.6923    0.4091    0.5143        22
                 Other     0.7241    0.7936    0.7573       344
    Project Management     0.8571    0.6154    0.7164        39
            Purchasing     1.0000    0.4000    0.5714        15
                 Sales  

In [21]:
# eval labels auf train labelset filtern
valid_s = set(le_s.classes_)
eval_s = eval_df[eval_df["seniority"].isin(valid_s)].dropna(subset=["seniority"]).copy()
print("Eval seniority used:", len(eval_s))

y_true = eval_s["seniority"].astype(str).values
y_true_ids = le_s.transform(y_true)

ds_eval_s = Dataset.from_dict({"text": eval_s["title"].astype(str).tolist(), "labels": y_true_ids.tolist()}).map(tokenize, batched=True)

pred = sen_trainer.predict(ds_eval_s)
pred_ids = np.argmax(pred.predictions, axis=-1)
y_pred = le_s.inverse_transform(pred_ids)

print("\n=== FINAL EVAL on eval_df (Seniority) ===")
print("Accuracy       :", accuracy_score(y_true, y_pred))
print("Macro Precision:", precision_score(y_true, y_pred, average="macro", zero_division=0))
print("Macro Recall   :", recall_score(y_true, y_pred, average="macro", zero_division=0))
print("Macro F1       :", f1_score(y_true, y_pred, average="macro", zero_division=0))
print("Weighted F1    :", f1_score(y_true, y_pred, average="weighted", zero_division=0))

print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4, zero_division=0))

preview = eval_s[["title", "seniority"]].copy()
preview["pred"] = y_pred
preview["correct"] = preview["seniority"] == preview["pred"]
display(preview.head(50))

Eval seniority used: 407


Map:   0%|          | 0/407 [00:00<?, ? examples/s]


=== FINAL EVAL on eval_df (Seniority) ===
Accuracy       : 0.7125307125307125
Macro Precision: 0.6133317907355843
Macro Recall   : 0.7202887700534759
Macro F1       : 0.622419340984148
Weighted F1    : 0.7319871062188262

Classification report:
              precision    recall  f1-score   support

    Director     0.5660    0.8824    0.6897        34
      Junior     0.2500    0.5000    0.3333        12
        Lead     0.9211    0.5600    0.6965       125
  Management     0.9172    0.7500    0.8252       192
      Senior     0.4124    0.9091    0.5674        44

    accuracy                         0.7125       407
   macro avg     0.6133    0.7203    0.6224       407
weighted avg     0.8148    0.7125    0.7320       407



,title,seniority,pred,correct
0,Prokurist,Management,Senior,False
1,CFO,Management,Management,True
3,Prokuristin,Management,Junior,False
4,CFO,Management,Management,True
17,Director expansión de negocio.,Director,Director,True
18,Gerente comercial,Lead,Junior,False
24,"APL-ansvarig, samordning",Lead,Lead,True
32,Kaufmännischer Leiter,Lead,Lead,True
34,Lab-Supervisor,Lead,Senior,False
42,Managing Director,Management,Director,False
